In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import math
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns  # visualization tool
from scipy import stats

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
path1 = "../input/marvel-superheroes/marvel_characters_info.csv"
path2 = "../input/pilot-sample/sp_dc.csv"
path3 = "../input/pilot-sample/sp_marvel.csv"

df = pd.read_csv(path1)
pilot_dc = pd.read_csv(path2)
pilot_marvel = pd.read_csv(path3)

df = df.loc[:, ['Name', 'Gender', 'Alignment', 'Publisher', 'Height', 'Race']]
new_df = df.dropna()
new_df = df[
    (df['Alignment'] == "good") &
    (df['Height'] > 0) &
    (df['Race'] == "Human")]

#new_df = df[
    #(df['Alignment'] == "good") &
    #(df['Height'] > 0) &
    #(df['Height'] < 300) &
    #(df['Gender'] == "Female") | 
    #(df['Gender'] == "Male")]

marvel_df = new_df[new_df['Publisher'] == "Marvel Comics"]
dc_df = new_df[new_df['Publisher'] == "DC Comics"]


# Apresentando o teste piloto

In [ ]:
pilot_marvel

In [ ]:
pilot_dc

### Verificar se os dados usados como teste piloto são válidos (podem ser considerados uma distribuição normal)
Aqui iremos fazer essa verificação usando o teste de Shapiro-Wilks ([veja mais](http://https://en.wikipedia.org/wiki/Shapiro–Wilk_test))

In [ ]:
# Shapiro wilks test para a amostra piloto da Marvel
stats.shapiro(pilot_marvel['Height'].to_numpy())

In [ ]:
# Shapiro wilks test para a amostra piloto da 
stats.shapiro(pilot_dc['Height'].to_numpy())

In [ ]:
frames = [pilot_marvel, pilot_dc]
data_merged = pd.concat(frames)
#stats.shapiro(data_merged['Height'].to_numpy())

## Calculando tamanho da amostra final

In [ ]:
#TESTE UNILATERAL

# Definindo o valor de alpha
alpha = 0.05 

# Médias das populações
mi_0 = pilot_marvel.mean()
mi_1 = pilot_dc.mean()

# Valores de sigma
sigma_0 = pilot_marvel.std()
sigma_1 = pilot_dc.std()

# poder do teste= 1 − β = 0.8
beta = 0.2

# Valores de z
z_alpha = stats.norm.ppf(1 - alpha)
z_beta = stats.norm.ppf(1 - beta)

# Calculando n
n = ((sigma_0**2 + sigma_1**2)*((z_alpha + z_beta)**2))/(abs(mi_1 - mi_0)**2)
n

In [ ]:
n = math.ceil(n)
n

# Como se dá a distribuição dos dados que temos

In [ ]:
# definindo nossas amostras
dc_df_sample = dc_df.sample(n=n)
dc_df_sample

In [ ]:
dc_df_sample.describe()

In [ ]:
marvel_df_sample = marvel_df.sample(n=n)
marvel_df_sample

In [ ]:
marvel_df_sample.describe()

In [ ]:
frames = [marvel_df_sample, dc_df_sample]
data_merged = pd.concat(frames)
data_merged.describe()

In [ ]:
#sns.catplot(y="Height",x="Publisher",kind="swarm",data=data_merged, hue="Gender")
sns.catplot(y="Height",x="Publisher",data=data_merged, hue="Publisher",palette="viridis")


In [ ]:
sns.catplot(y="Height",x="Publisher",jitter=False,data=data_merged, hue="Gender", palette="rocket")

In [ ]:
sns.catplot(y="Height",x="Publisher",kind="box",data=data_merged, palette="viridis")

In [ ]:
#sns.histplot(x="Height", hue="Publisher",kde=True,data=data_merged, palette="rocket")
sns.histplot(x="Height", hue="Publisher",kde=True,data=data_merged, palette="viridis", bins=15)

# Análise Inferencial
# Teste de Hipótese

> Premissa: "Heróis da Marvel são menores que os heróis da DC."

***Teste unilateral*** <br>
mi_0 -> marvel characters height <br>
mi_1 -> dc characters height<br>

H0 : mi_0 >= mi_1<br>
H1 : mi_0 < mi_1<br>

***Perguntas a serem respondidas*** <br>
Realiza-se o teste z ou teste t? teste t<br>
As amostras são pareadas ou não pareadas? Não pareadas<br>
O teste é bilateral ou unilateral? Unilateral <br> 
As variâncias são semelhantes? as variâncias da população não são iguais<br>

## Para começar: nosso teste é paramétricos ou não-paramétricos?
1.  Testes paramétricos - baseiam-se na hipótese de que as variáveis que representam os grupos que estão sendo comparados apresentem uma distribuição mais ou menos bem comportada;
2.  Teste não-paramétricos - não é necessário supor que a variável em análise apresenta um determinado tipo de distribuição

3. Para testes paramétricos:
-> É necessário confirmar que a variável possui determinada função de probabilidade (prova de aderência); 
-> Quando a distribuição é a Normal, a prova de aderência se denomina teste de normalidade; 
-> Exemplos de testes de normalidade: Teste de Kolmogorov-Smirnov (K-S) e Teste de Shapiro-Wilks (W)

4. Fazendo o teste Shapiro-Wilks
-> O teste de Shapiro-Wilk testa a hipótese nula de que os dados foram extraídos de uma distribuição normal 
-> A chance de rejeitar a hipótese nula quando verdadeira é próxima a 5%, independentemente do tamanho da amostra. 

In [ ]:
print(marvel_df_sample['Height'].to_numpy())
print(dc_df_sample['Height'].to_numpy())

In [ ]:
print(stats.shapiro(marvel_df_sample['Height'].to_numpy()))
print(stats.shapiro(dc_df_sample['Height'].to_numpy()))


### Resultado: Teste paramétrico
1. Hipótese H0 <br>
Como o valor de p> α, aceitamos H0.
Presume-se que os dados são normalmente distribuídos.
Em outras palavras, a diferença entre a amostra de dados e a distribuição normal não é grande o suficiente para ser estatisticamente significativa.


## Teste t - Amostras independentes
Dada duas amostras extraídas das populações A e B, o valor obtido para o
t do teste indica a diferença entre as amostras.


### 1. Teste das variâncias
Um teste F é usado para testar se duas variâncias populacionais são iguais. As hipóteses nula e alternativa para o teste são as seguintes:<br>
H0: σ1^2 = σ2^2 (as variâncias da população são iguais)<br>
H1: σ1^2 ≠ σ2^2 (as variâncias da população não são iguais)<br>

In [ ]:
var_a = var_b = 1
gln = gld = n - 1

if(marvel_df_sample['Height'].std() > dc_df_sample['Height'].std()):
    var_a = marvel_df_sample['Height'].std()
    var_b = dc_df_sample['Height'].std()
    mean_a = marvel_df_sample['Height'].mean()
    mean_b = dc_df_sample['Height'].mean()
else:
    var_a = dc_df_sample['Height'].std()
    var_b = marvel_df_sample['Height'].std()
    mean_a = dc_df_sample['Height'].mean()
    mean_b = marvel_df_sample['Height'].mean()

F_teste = var_a/var_b

p_value = stats.f.cdf(F_teste, gln, gld)

if p_value > alpha:
    print("Rejeita a hipótese nula")
else:    
    print("Aceita a hipótese nula")

In [ ]:
#Grau de liberdade

gl_numerador = ((((var_a**2)/n) + ((var_b**2)/n))**2)
gl_denominador = ((((var_a **2)/n)**2)/(n**-1)) + ((((var_b**2)/n)**2)/(n**-1))
gl = gl_numerador/gl_denominador
print(gl)

### 2. Valor do teste t

In [ ]:
# Valor do teste t


s_a_b = math.sqrt(((n - 1)*(var_a**2) + (n - 1)*(var_b**2))/(n + n - 2))
t_numerador = mean_a - mean_b
t_denominador = math.sqrt((1/n)+(1/n))*s_a_b
t_teste = t_numerador/t_denominador
t_tabela = stats.ttest_ind(marvel_df_sample['Height'].to_numpy(), dc_df_sample['Height'].to_numpy())

if(t_teste > t_tabela.statistic ):
    print("Rejeita H0, com um nível de significância de 0,05")
    print("Isso implica que os heróis da Marvel são menores que os heróis da DC")
else:
    print("Aceita H0, com um nível de significância de 0,05")
    print("Isso implica que os heróis da Marvel são maiores ou tem o mesmo tamanho dos heróis da DC")